In [80]:
import os
import requests
import json
import csv
import pandas as pd
from dotenv import load_dotenv
from time import sleep
load_dotenv()

from pybliometrics.scopus.utils import config
# print(config['Authentication']['APIKey'])  # Show keys
config['Authentication']['APIKey'] = "7f59af901d2d86f78a1fd60c1bf9426a"
print(config['Authentication']['APIKey'])

7f59af901d2d86f78a1fd60c1bf9426a


In [81]:
from pybliometrics.scopus import CitationOverview, AuthorRetrieval, AuthorSearch

In [82]:
PATH_TO_INPUT_CSV = "in/PSYT faculty member lists UTF8.csv"
OUTPUT_FILEPATH = "out/PSYT faculty member lists SciVal formatted.csv"
cols = ['Last Name', 'First Name']
subjects_list = ["NEUR", "BIOC", "MEDI", "PSYC", "HEAL", "IMMU", "NURS", "PHAR"]
affiliation_list = ["The University of British Columbia", "Djavad Mowafaghian Centre for Brain Health", "UBC Hospital", "BC Children's Hospital Research Institute", "University of British Columbia Okanagan", "University of British Columbia, Faculty of Medicine",]
city_list = ["Vancouver"]
country_list = ["Canada"]

In [83]:
member_arr = []
with open(PATH_TO_INPUT_CSV, "r") as f:
    r = csv.reader(f)
    for row in r:
        if row[0] != "":
            member_arr.append(row)

# skip header rows
member_arr = member_arr[1:]
print(len(member_arr))

802


In [84]:
# member_arr = [["Abdel-Fattah","Mohamed"]]

In [93]:
last_names = []
first_names = []
affils = []
auth_ids = []
orcids = []
tags = []
cities = []
countries = []

warnings = []
subjects = []

no_search_results = []

#go through all members
for member in member_arr:
    start = 0
    sleep(1) #pause for one second to prevent API warning

    #format first and last names
    lastname = member[0].strip()
    firstname = member[1].strip()

    print("searching for " + firstname + " " + lastname)

    query = f"authlastname({lastname}) and authfirst({firstname})"
    s = AuthorSearch(f"authlast({lastname}) and authfirst({firstname})")
    
    warning = ""
    authors = s.authors
    num_authors_found = s.get_results_size()
    
    #returns number of authors found
    
    if num_authors_found == 0:
        print("no scival profile found for " + firstname + " " + lastname + "\n")
        last_names.append(lastname)
        first_names.append(firstname)
        affils.append("")
        cities.append("")
        countries.append("")
        auth_ids.append("")
        warnings.append("no profile found")
        subjects.append("")
        orcids.append("")
        no_search_results.append(member)
        continue

    author_idx = 0
    match_found = False
    #multiple authors found with first/last name search
    if num_authors_found >= 1:
        print(str(num_authors_found) + " profile(s) found for " + firstname + " " + lastname)

        if num_authors_found > 1:
            warning = warning + "multiple profiles; "
        
        best_match = 0
        #look through all the authors
        for idx in range(num_authors_found):
            print("looking at profile " + str(idx))

            # run comparison checks
            subject_check = False
            affiliation_check = False
            city_check = False
            country_check = False

            try:
                subject_check = any([topic in authors[idx].areas for topic in subjects_list])
            except:
                print("no subject with associated profile")

            try:
                affiliation_check = any([affil in affiliation_list for affil in authors[idx].affiliation])
                city_check = any([city in city_list for city in authors[idx].city]) 
                country_check = any([country in affiliation_list for country in authors[idx].country])
            except:
                print("no affiliation with associated profile")

            # if subject and affiliations match
            if subject_check and affiliation_check:
                print("subject and affiliation match")
                best_match = idx 
                match_found = True
                break

            if affiliation_check:
                print("affiliation match")
                best_match = idx 
                match_found = True
                break
            
            if (city_check or country_check) and subject_check:
                print("city/country match")
                best_match = author_idx
                match_found = True
                break
        
    if not match_found:
        print("looked through all profiles. no scival profile found for " + firstname + " " + lastname + "\n")
        last_names.append(lastname)
        first_names.append(firstname)
        affils.append("")
        cities.append("")
        countries.append("")
        auth_ids.append("")
        warnings.append(warning)
        subjects.append("")
        orcids.append("")
        no_search_results.append(member)
        continue

    else:
        author_idx = best_match
        
    
    #get author affiliation name
    affil_name = ""
    city = ""
    country = ""
    try:
        affil_name = authors[author_idx].affiliation
        city = authors[author_idx].city
        country = authors[author_idx].country
        if affil_name not in affiliation_list:
            warning = warning + "affiliation not in list"
    except:
        warning = "no affiliation"
    
    #get author scopus id
    author_id = ""
    try:
        author_id = authors[author_idx].eid.split("-")[-1]
    except:
        pass
    
    # affil_id = data["search-results"]["entry"][0]["affiliation-current"]["affiliation-id"]

    #get author publication topics
    subject = ""
    try:
        subject = authors[author_idx].areas
        if not any([topic in authors[author_idx].areas for topic in subjects_list]):
            warning = warning + " subject mismatch; "
    except:
        warning = warning + " no subjects found; "
        pass

    #get author orcid if available
    orcid = ""
    try:
        orcid = authors[author_idx].orcid
    except:
        pass
    
    #add variables to respective arrays to turn into csv
    try:
        scopus_lastname = authors[author_idx].surname
        scopus_firstname = authors[author_idx].givenname
    except:
        scopus_lastname = lastname
        scopus_firstname = firstname

    last_names.append(scopus_lastname)
    first_names.append(scopus_firstname)
    affils.append(affil_name)
    cities.append(city)
    countries.append(country)
    auth_ids.append(author_id)
    warnings.append(warning)
    subjects.append(subject)
    orcids.append(orcid)

    print("\n")

searching for Pieter Aartsma
no scival profile found for Pieter Aartsma

searching for Mohamed Abdel-Fattah
21 profile(s) found for Mohamed Abdel-Fattah
looking at profile 0
looking at profile 1
looking at profile 2
looking at profile 3
looking at profile 4
looking at profile 5
no affiliation with associated profile
looking at profile 6
looking at profile 7
looking at profile 8
looking at profile 9
looking at profile 10
looking at profile 11
looking at profile 12
looking at profile 13
looking at profile 14
looking at profile 15
looking at profile 16
looking at profile 17
looking at profile 18
looking at profile 19
looking at profile 20
looked through all profiles. no scival profile found for Mohamed Abdel-Fattah

searching for Ozotu Abu
no scival profile found for Ozotu Abu

searching for Trudy Jean Adam
no scival profile found for Trudy Jean Adam

searching for Leona Adams
no scival profile found for Leona Adams

searching for Adetokunbo Adeshina
1 profile(s) found for Adetokunbo Ades

KeyboardInterrupt: 

In [90]:
out_dict = {
    "Last Name": last_names,
    "First Name": first_names,
    "Affiliation": affils,
    "Country": countries,
    "Scopus Author ID": auth_ids,
    "ORCID": orcids,
    "Research Areas": subjects,
    "Warnings": warnings,
}
print(no_search_results)

for key in out_dict:
    print(len(out_dict[key]))

[['Aartsma', 'Pieter'], ['Abdel-Fattah', 'Mohamed'], ['Abu', 'Ozotu'], ['Adam', 'Trudy Jean'], ['Adams', 'Leona'], ['Adeshina', 'Adetokunbo'], ['Afridi', 'Qasim'], ['Afzal', 'Ayesha'], ['Afzal', 'Neelam'], ['Agafonov', 'Marina'], ['Agbahovbe', 'Onome'], ['Agboji', 'Hezekiah'], ['Agranovich', 'Eugene'], ['Ahmad', 'Nauman'], ['Ainsworth', 'Nick'], ['Ainsworth', 'Geoffrey'], ['Ali', 'Shaik'], ['Allen', 'Katie'], ['Allibhai', 'Fatima'], ['Almas', 'Naveed'], ['Amarjothi Nagendran', 'Sreedharan'], ['Anderson', 'Cameron'], ['Anwar', 'Muhammad'], ['Anyansi', 'Tochukwu'], ['Argouarch', 'Sylvie'], ['Armstrong', 'Kimberly'], ['Au', 'Chi Hung'], ['Au', 'Raymond'], ['Austin', 'Jehannine'], ['Awe', 'Ayodeji'], ['Aydin', 'Cristina'], ['Azarbar', 'Ataa'], ['Azim', 'Hassan'], ['Baer', 'Susan'], ['Bailey', 'Anthony'], ['Bains', 'Ramnique'], ['Balakrishnan', 'Pradeep'], ['Ballance', 'Dina'], ['Banno', 'Brian'], ['Barale', 'Anthony'], ['Bardell', 'Andrea'], ['Barley', 'Jennifer'], ['Bates', 'Alan Thomas']

In [91]:
df = pd.DataFrame.from_dict(out_dict)
df.to_csv("strictly_affils4.csv")
# df.to_csv(OUTPUT_FILEPATH)